In [ ]:
#Data.csv

**Step 1: Importing the libraries**

In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

**Step 2: Importing dataset**

In [106]:
data=pd.read_csv('Data.csv')
data

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
6,Spain,NaN,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [116]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Country    10 non-null     object 
 1   Age        9 non-null      float64
 2   Salary     9 non-null      float64
 3   Purchased  10 non-null     object 
dtypes: float64(2), object(2)
memory usage: 448.0+ bytes


**Step 3: Handling the missing data**

To fill missing values either we can go by median approach or make NaN value as test data and remaing as training data,
build a model and predict the value and fill it in missing values.

By following median approach we can fill Nan values as below

In [108]:
age_nan=data.groupby('Country')['Age'].median()
salary_nan=data.groupby('Country')['Salary'].median()
print(age_nan)
print(salary_nan)

Country
France     40.5
Germany    40.0
Spain      32.5
Name: Age, dtype: float64
Country
France     69500.0
Germany    68500.0
Spain      52000.0
Name: Salary, dtype: float64


So,age_nan can be filled with 32.5 and salary_nan can be filled with 68500

In [111]:
df=data.drop(6,axis=0)
df

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [112]:
Purchased=pd.get_dummies(df['Purchased'],drop_first=True)
Country=pd.get_dummies(df['Country'],drop_first=True)
df.drop(['Country','Purchased'],axis=1,inplace=True)
df=pd.concat([df,Country,Purchased],axis=1)
df

,Age,Salary,Germany,Spain,Yes
0,44.0,72000.0,0,0,0
1,27.0,48000.0,0,1,1
2,30.0,54000.0,1,0,0
3,38.0,61000.0,0,1,0
4,40.0,NaN,1,0,1
5,35.0,58000.0,0,0,1
7,48.0,79000.0,0,0,1
8,50.0,83000.0,1,0,0
9,37.0,67000.0,0,0,1


In [117]:
X=df.drop('Salary',axis=1)
Y=df.Salary
X_train=X.drop(4,axis=0)
X_test=X.iloc[4:5,:]
Y_train=Y.dropna()
Y_test=Y.loc[4]

In [118]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(X_train,Y_train)

LinearRegression()

In [119]:
pred=reg.predict(X_test)
pred

array([70683.56251999])

In [120]:
print(reg.coef_)
print(reg.intercept_)

[ 1438.75919412  2576.43108411 -1724.65622002  2183.56251999]
8373.201151263223


Salary NaN value can be filled as 70683.56251999
Age NaN value can be filled as 31.5(got this value by substituting values in the equation derived from the above model)

In [126]:
data=data.fillna({'Age':31.5,'Salary':70683.0})
data

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,70683.0,Yes
5,France,35.0,58000.0,Yes
6,Spain,31.5,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


**Step 4: Encoding categorical data**

In [127]:
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
dfle=data.copy()
dfle['Country']=LE.fit_transform(dfle.Country)
dfle['Purchased']=LE.fit_transform(dfle.Purchased)
dfle

,Country,Age,Salary,Purchased
0,0,44.0,72000.0,0
1,2,27.0,48000.0,1
2,1,30.0,54000.0,0
3,2,38.0,61000.0,0
4,1,40.0,70683.0,1
5,0,35.0,58000.0,1
6,2,31.5,52000.0,0
7,0,48.0,79000.0,1
8,1,50.0,83000.0,0
9,0,37.0,67000.0,1


**Step 5: Creating a dummy variable**

In [131]:
df1=data.copy()
Purchased=pd.get_dummies(df1['Purchased'],drop_first=True)
Country=pd.get_dummies(df1['Country'],drop_first=True)
df1.drop(['Country','Purchased'],axis=1,inplace=True)
df1=pd.concat([df1,Country,Purchased],axis=1)
df1

,Age,Salary,Germany,Spain,Yes
0,44.0,72000.0,0,0,0
1,27.0,48000.0,0,1,1
2,30.0,54000.0,1,0,0
3,38.0,61000.0,0,1,0
4,40.0,70683.0,1,0,1
5,35.0,58000.0,0,0,1
6,31.5,52000.0,0,1,0
7,48.0,79000.0,0,0,1
8,50.0,83000.0,1,0,0
9,37.0,67000.0,0,0,1


**Step 6: Splitting the datasets into training sets and Test sets**

In [134]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(dfle.drop('Purchased',axis=1),dfle['Purchased'],test_size=0.2,random_state=5)

**Step 7: Feature Scaling**

In [135]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit_transform(X_train)
scaler.transform(X_test)

array([[-1.44115338, -0.19594137,  0.16679314],
       [-1.44115338, -0.44674633, -0.56919424]])